In [ ]:
using Pkg
Pkg.activate("../Tambo/")
using Tambo

In [ ]:
using Plots
using JLD2
using StaticArrays
using StatsBase
include("../figures/paperstyle.jl")

In [ ]:
config_f = jldopen("../resources/example_config.jld2")
const config = SimulationConfig(; Dict(k=>v for (k, v) in config_f["config"] if k != :geo_spline_path)...)
const geo = Tambo.Geometry(config)
const plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
const altmin = 1.8925255158436627units.km
const altmax = 4.092525515843662units.km

In [ ]:
jldf = jldopen("../resources/example_hit_maps.jld2")
key = "2000.0_100.0"
hit_maps = jldf[key]

array_length = parse(Float64, split(key, '_')[1])
spacing = parse(Float64, split(key, '_')[2])

modules = Tambo.make_detector_array(
    whitepaper_coord,
    array_length * units.m,
    spacing * units.m,
    altmin,
    altmax,
    plane,
    geo
);

In [ ]:
function find_time_range(hit_map)
    tmin, tmax = Inf, -Inf
    for (idx, evts) in hit_map
        for evt in evts
            tmin = minimum([evt.time, tmin])
            tmax = maximum([evt.time, tmax])
        end
    end
    return tmin, tmax
end

In [ ]:
function plot_topographic_map!(
    plt,
    geo;
    xmin=-2.5units.km,
    xmax=2.5units.km,
    ymin=-2.5units.km,
    ymax=2.5units.km,
    contour_color=palette(:lapaz)
    
)
    
    contour_xs = LinRange(xmin, xmax, 201)
    contour_ys = LinRange(ymin, ymax, 200)
    plot!(
        xlimits=(xmin, xmax)./units.km,
        ylimits=(ymin, ymax)./units.km,
        colorbar_title="Altitude [km]",
        xlabel=L"x~\left[\mathrm{km}\right]",
        ylabel=L"y~\left[\mathrm{km}\right]"
    )
    contour!(
        plt,
        contour_xs ./ units.km,
        contour_ys ./ units.km,
        @. (geo(contour_xs', contour_ys) + geo.tambo_offset[3]) / units.km;
        fill=true,
        color=contour_color,
        clims=(-1.5+geo.tambo_offset[3]/units.km, 1.4+geo.tambo_offset[3]/ units.km)
    )
end

function plot_topographic_map(
    geo;
    xmin=-2.5units.km,
    xmax=2.5units.km,
    ymin=-2.5units.km,
    ymax=2.5units.km,
    contour_color=palette(:lapaz)
)
    plot!(
        plt,
        size=(500, 500),
    )
    plot_topographic_map!(plt, geo, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax, contour_color=contour_color)
    return plt
end



In [ ]:
function plot_modules!(
    plt,
    modules,
    hit_map=nothing;
    time_cmap=cgrad(:rainbow, rev=true),
    module_color=colorant"black",
    tcut::Float64=Inf
)

    colors = fill(module_color, length(modules))
    markersize = fill(3.0, length(modules))
    
    if ~(hit_map==nothing)
        tmin, tmax = find_time_range(hit_map)
        hit_color(t) = get(time_cmap, (t - tmin) / (tmax - tmin))
        ts = LinRange(tmin, tmax, 100)
        for (idx, evts) in hit_map
            evts = filter(e->e.time<tcut, evts)
            if length(evts)==0
                continue
            end
            colors[idx] = hit_color(mean(getfield.(evts, :time)))
            markersize[idx] = 3 + log(length(evts))
        end
    end

    scatter!(
        plt,
        getindex.(getfield.(modules, :pos), 1) ./ units.km,
        getindex.(getfield.(modules, :pos), 2) ./ units.km,
        alpha=0.9,
        markersize=markersize,
        color=colors,
        markerstrokewidth=0,
        label=""
    )
end

function plot_modules(
    modules,
    hit_map=nothing;
    time_cmap=cgrad(:rainbow, rev=true),
    module_color=colorant"black",
    tcut::Float64=Inf
)
    plt = plot()
    plot_modules!(
        plt,
        modules,
        hit_map=hit_map,
        time_cmap=time_cmap,
        module_color=module_color,
        tcut=tcut,
    )
    return plt
end

In [ ]:
event_number = 17452
hit_map = hit_maps[string(event_number)]
plt = plot(background_color=nothing, size=(500, 600))
plot_topographic_map!(
    plt,
    geo,
    contour_color=cgrad(:grayyellow, rev=false),
    # contour_color=palette(:linear_grey_0_100_c0_n256, rev=true),
    xmin=-2units.km,
    xmax=2units.km
)
plot_modules!(
    plt,
    modules,
    hit_map,
    # time_cmap=cgrad(:rainbow, rev=true)
    time_cmap=cgrad(:matter, rev=false)
)
savefig(plt, "../figures/neutrino_2024_event_display.pdf")
display(plt)

In [ ]:
cgrad(:coffee, 20, categorical=true, rev=false)

In [ ]:
cgrad(:matter, rev=false)

In [ ]:
jldf = jldopen("/Users/jlazar/Downloads/nhits_nmodules.jld2")

In [ ]:
10 .^ (0:0.25:1)

In [ ]:
maximum(jldf["nhits"])

In [ ]:
bins1 = [0, 1, 3, 6, 10, 18, 30, 56, 100, 180]
bins2 = [0, 1, 3, 6, 10, 18, 30, 56, 100, 180, 316, 562, 1000, 1778, 3162, 5623, 10000, 17783, 31622, 63835]


h = fit(Histogram, (jldf["nmodules"], jldf["nhits"]), (bins1, bins2))

In [ ]:
vcat([-0.3], log.(10, bins2[2:2:end]))

In [ ]:
(vcat([-0.3], log.(10, bins2[2:2:end])), vcat(["0"], [string(x) for x in bins2[2:2:end]]))

In [ ]:
cgrad(:matter, 20, categorical = true)

In [ ]:
cents1 = (bins1[1:end-1] + bins1[2:end]) / 2
cents2 = (bins2[1:end-1] + bins2[2:end]) / 2


plt = plot(
    size=(450, 400),
    xlimits=(-0.3, 4.3),
    ylimits=(-0.3, log(10, 180)),
    colorbar_title=L"\log_{10}(N_{\mathrm{evts}})",
    yticks = (vcat([-0.3], log.(10, bins1[2:end])), [string(x) for x in bins1]),
    xticks = (vcat([-0.3], log.(10, bins2[2:2:end-1])), vcat(["0"], [string(x) for x in bins2[2:2:end-1]])),
    xlabel=L"N_{\mathrm{hits}}",
    ylabel=L"N_{\mathrm{modules}}",
    leftmargin=2mm
    # background_color=nothing

)
heatmap!(plt, log.(10, cents2), log.(10, cents1), log.(10, h.weights), c=cgrad(:matter, 21, categorical = true, rev=true))
# plot!(
#     plt,
#     [log(10, 33), log(10, 33)],
#     log10.([minimum(cents1), 180]),
#     color=:black
# )
# plot!(
#     plt,
#     log10.([minimum(cents2), maximum(cents2)]),
#     [log(10, 3), log(10, 3)],
#     color=:black
# )

# heatmap!(plt, lg.(10, cents2), log.(10, cents1), log.(10, h.weights))
savefig(plt, "../figures/neutrino_2024_nhits_nmodules.pdf")
display(plt)

In [ ]:
include("../scripts/trigger_defs.jl")